# Information Warfare: Russia’s use of Twitter during the 2016 US Presidential Election (Work in Progress)

### Analysis by Ben Forleo

### Part 1

#### Background and research question

On January 7, 2017 the Office of the Director of National Intelligence released a declassified report detailing a Russian campaign to influence the 2016 US presidential election.  The report described a broad range of activities aimed at denigrating Secretary Hillary Clinton and undermining faith in the US democratic process. One facet of this campaign involved social media “trolls” employed by the Internet Research Agency, an organization financed by a close Putin ally with ties to Russian intelligence.  In an indictment of the Internet Research Agency dated February 16, 2018, special counsel Robert Mueller III accused this organization of attempting to sow discord online by posting derogatory and false information with an intent to interfere in the US 2016 election.

On August 27, 2018, the data journalism organization FiveThirtyEight published 2,973,371 tweets from accounts identified by Twitter as being associated with the Internet Research Agency. The dataset is the product of work done by Clemson Univeristy Professors Darren Linville and Patrick Warren who collected the data from Clemson University's Social Media Listening center. The researchers also classifed accounts according to their language and behavior: categories include Right Troll, Left Troll, News Feed, Hashtag Gamer, and Fearmonger. 

In a [working paper](http://pwarren.people.clemson.edu/Linvill_Warren_TrollFactory.pdf) describing their work, Linville and Warren discuss the qualitiative procedure that they employed to label accounts.

The purpose of this project is to test whether Linville and Warren's account categorization scheme can be replicated quantitatively using unsupervised learning techniques.


#### Procedure

The procedure that I will employ is as follows:

- Clean and tokenize text using the spaCy library
- Create vectorized representations of tweets using the doc2vec algorithm and the gensim library
- Cluster the vector representation of tweets using KMeans and Gaussian mixture models. 
- Compare the results of these algorithmic labeling methods with the labels assigned by Linville and Warren.


Let's import some data!

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

import spacy
import os
import pickle

from collections import Counter

from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py

init_notebook_mode(connected=True)

df = pd.read_pickle('/Users/benjaminforleo/disinformation_project_data/labeled_tweets.pkl')
df.reset_index(drop = True, inplace = True)

As always, we need to perform some EDA to get a sense of what's in our datset. Since I don't speak Russian, I will filter to include only English language tweets. 

In [2]:
# Filter to include only English Tweets
df = df[df.language == "English"]

print("Shape: ", df.shape, "\n", "\n"
     "# Unique accounts: ", len(set(df.author)), "\n", "\n",
     "Account Type: ", list(set(df.account_type)), "\n", "\n",
     "Account Category: ", list(set(df.account_category)))

Shape:  (2116867, 21) 
 
# Unique accounts:  2161 
 
 Account Type:  ['German', 'French', 'Left', 'Commercial', 'Arabic', 'Ebola ', 'news', 'Portuguese', 'Koch', 'local', 'ZAPOROSHIA', 'Spanish', 'Russian', 'Hashtager', '?', 'Italian', 'Right'] 
 
 Account Category:  ['Unknown', 'HashtagGamer', 'NewsFeed', 'RightTroll', 'NonEnglish', 'LeftTroll', 'Fearmonger', 'Commercial']


There are two sets of labels that Linville and Warren have created, namely, account type and account name. Account type consists of more granular labels, while account category is more broad. For this exercise, I will focus on the account category set of labels. 

Before we move on, I should point out that the labeling scheme employed by Linville and Warren allows for accounts that primarily tweet in English to be labeled as Non English. For example, an account that primarily tweets about international events, such as the war in Ukraine, would be labeled as Non English, even if the text is in English. The graph below indicates that activity characterized by this behavior is limited. 

In [3]:
counts_by_type = dict(Counter(df.account_category))

account_type = list(counts_by_type.keys())
values = list(counts_by_type.values())

account_type = [x for _,x in sorted(zip(values ,account_type), reverse = True)]
values = sorted(values, reverse = True)

other_color = ['Unknown', 'Commercial', 'NonEnglish', 'Fearmonger']

color_list = ['rgb(175, 88, 141)' if i in other_color else 'rgb(23,62,90)' for i in account_type]

data = [go.Bar(
            x=account_type,
            y=values,
            marker = dict(color = color_list),
            opacity = .7
    )]

layout = go.Layout(title = dict(text = 'Counts of Tweets by Category', font = dict(size = 30)), 
                   xaxis = dict(
                       tickangle = 45, 
                       tickfont = dict(size = 15),
                       automargin = True),
                  yaxis = dict(title = dict(text = 'Tweets', font = dict(size = 20))))

fig = go.Figure(data = data, layout = layout)

py.iplot(fig, filename='basic-bar')

The previous chart displayed information for overall tweets. However, what we are really interested in is activity per account. That is, it would be nice to know how many right trolls are in the dataset vs. left trolls and so on. Let's go ahead and visualize this information now. 

In [4]:
account_analysis = df.groupby(['author', 'account_category'])['content'].count().reset_index()

account_analysis = Counter(account_analysis.account_category)

account_type = list(account_analysis.keys())
values = list(account_analysis.values())

account_type = [x for _,x in sorted(zip(values ,account_type), reverse = True)]
values = sorted(values, reverse = True)

color_list = ['rgb(175, 88, 141)' if i in other_color else 'rgb(23,62,90)' for i in account_type]

data = [go.Bar(
            x=account_type,
            y=values,
            marker = dict(color = color_list),
            opacity = .7
    )]

layout = go.Layout(title = dict(text = 'Counts of Accounts by Category', font = dict(size = 30)), 
                   xaxis = dict(
                       tickangle = 45, 
                       tickfont = dict(size = 15),
                       automargin = True),
                  yaxis = dict(title = dict(text = 'Accounts', font = dict(size = 20))))

fig = go.Figure(data = data, layout = layout)

py.iplot(fig, filename='basic-bar')

Ok, this is an interesting chart. While Non English tweets account for very little of the overall tweet volume, there is a sizable number of Non English accounts in the dataset. 

For our procedure to work, we need to make sure that we have enough information (tweets) for each account for doc2vec to appropriatly embed our text in a vector space. As such, we need to ensure that we only consider accounts that meet some threshold for a minimum number of tweets, which we we do below. 

In [5]:
# Let's get counts of the number of tweets by each author
counts_by_author = df[['author', 'content']].groupby('author').count()

counts_by_author.reset_index(inplace = True)

print(sum(counts_by_author.content > 400))

author_series = counts_by_author.author[counts_by_author.content > 400]

df = df[df.author.isin(author_series)]

463


It should be noted that the choice of 400 tweets is arbitrary. Notice that by setting this threshold, the amount of Non English accounts in the dataset has decreased considerably. 

In [6]:
account_analysis = df.groupby(['author', 'account_category'])['content'].count().reset_index()
account_analysis = Counter(account_analysis.account_category)

account_type = list(account_analysis.keys())
values = list(account_analysis.values())

account_type = [x for _,x in sorted(zip(values ,account_type), reverse = True)]
values = sorted(values, reverse = True)

color_list = ['rgb(175, 88, 141)' if i == 'NonEnglish' else 'rgb(23,62,90)' for i in account_type]

data = [go.Bar(
            x=account_type,
            y=values,
            marker = dict(color = color_list),
            opacity = .7
    )]

layout = go.Layout(title = dict(text = 'Accounts with more than 400 Tweets', font = dict(size = 30)), 
                   xaxis = dict(
                       tickangle = 45, 
                       tickfont = dict(size = 15),
                       automargin = True),
                  yaxis = dict(title = dict(text = 'Accounts', font = dict(size = 20))))

fig = go.Figure(data = data, layout = layout)

py.iplot(fig, filename='basic-bar')

# Sampling and Pre-Processing

As you may know, natural language processing can be computationally expensive. Since we are dealing with a large number of tweets, it makes sense to take a sample of the data that we want to work with. 

Here, I take a 30% sample of tweets from each account. 

Note: the 30% sample size is arbitrary.

In [7]:
df.content = df.content.astype(str)

# Create the groupby object of author and category. Note: every author is assigned one category
df_sampled = df.groupby(['author', 'account_category'])['content']

# Take a 30% sample from each group
df_sampled = df_sampled.apply(lambda x: x.sample(frac=0.3, replace = False)).reset_index()

You might be inclined to ask: why take a sample per account rather than just a sample of the overall dataset? When I first began expirimenting with this data, I tried to embed each individual tweet in a vector space. However, this procedure produced nonsensical results. It seems that there is not enough information contained in an individual tweet for us to gleen any useful unsights about the author, or discriminate between authors. 

However, if we were to combine tweets per author such that we have a single document for each author that is representative of everything that a particular author has ever tweeted, we will have more than enough information to give doc2vec a chance to work. 

There is just one problem: spaCy breaks if we try to parse long documents of text. As such, we will need to group our tweets by author after running the data through spaCy. 

Before we get to spaCy, we need to do some minor preprocessing. Here, I remove hyperlinks and and strip redundant whitepace from the text. I also expirimented with removing the RT symbol, but this did not seem to have any meaningful impact on our analysis. 

In [8]:
# SAMPLE AND PRE-CLEANING

# REMOVE RT SYMBOL
#from processing_functions import rt_remover
#df_sampled.content = df_sampled.content.apply(rt_remover)

# REMOVE ANY HYPERLINKS
from processing_functions import link_remover
df_sampled.content = df_sampled.content.apply(link_remover)


# STRIP ANY WHITESPACE ON EITHER SIDE OF THE TEXT
df_sampled.content = df_sampled.content.str.strip()

Now that we have done some basic pre-processing, we can begin to parse and clean our text with spaCy.

# Tokenization, cleaning, and lemmatization with spaCy

- Note: this takes approximately 5 - 10 minutes to run.

In [9]:
from spacy.tokens import Token

# This allows us to add custom attributes to tokens, in this case, hashtags and accounts
Token.set_extension('is_hashtag', default = False, force = True)
Token.set_extension('is_account', default = False, force = True)

# These functions tell spaCy what should be considered a hashtage or account
from processing_functions import hashtag_pipe
from processing_functions import is_account_pipe

# We can  disable pipeline objects to save time: disable = ['parser', 'etc']
nlp = spacy.load("en", disable = ['parser', 'ner'])

# Here I add the two custom functions for hashtags and accounts to the pipeline
nlp.add_pipe(hashtag_pipe)
nlp.add_pipe(is_account_pipe)

# And we're off!
parsed_tweets = list(nlp.pipe(df_sampled.content))

# Gensim


Before we train our gensim doc2vec model, we need to do some more cleaning. The clean_doc function is located in the processing_functions.py file in this repository. 

Note: While I tried running the following without lemmatizing or removing stop words, I did not find that this made any meaningful difference in terms of the clusters that do or do not form.

In [10]:
from processing_functions import clean_doc
from processing_functions import clean_doc_no_lemma

lemma = list(map(clean_doc, parsed_tweets))
#no_lemma = list(map(clean_doc_no_lemma, parsed_tweets))

The next step is perhaps the most critical to this analysis. We need to group the parsed and cleaned tweets by author, so that for each author, we have a 30% sample of everything that they have ever tweeted in a single list (as discussed above). We can easily do this by applying our own function to a Pandas groupby object. 

In [11]:
# each row in parsed content is a list of lists
df_sampled['parsed_content'] = lemma

# lets flatten these so that each row only has one list
from processing_functions import group_lists
        
df_grouped = df_sampled.groupby(['author', 
                    'account_category'])['parsed_content'].apply(group_lists).reset_index()

# Convert the parsed content Series to a list for gensim
parsed_content = list(df_grouped.parsed_content)

Now that out data is clean and in the proper format, we can go ahead and create our gensim doc2vec model. It never ceases to amaze me that we can implement such a poweful algorithm in just a few lines of code. 

In [12]:
# create a doc2vec model
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# parsed_content is the list of parsed text
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(parsed_content)]

# Vector size of 300
model = Doc2Vec(documents, vector_size=300, window=5, min_count=3, workers=6)

arr_list = []

for index in range(0, len(model.docvecs)):
    arr_list.append(model.docvecs[index])
    
vec_array = np.stack(arr_list)

# Visualization and clustering

Great! We've cleaned, tokenized, and embeded our textual data in a vector space. Let's try to visualize our data and see where we are at. 

Since we are working with a 300 dimension dataset, we need to employ a dimension reduction technique if we want to visualize some of the patterns in our data. t-SNE is a great algorithm for this purpose, and we will employ this technique in the space below. 

By reducing the number of dimensions with t-SNE, we can visualize our data on a two-dimensional canvas. In the following chart the color of each individual datapoint will correspond to a label manually assigned by Professors Warren and Linville. 

In [13]:
# Instantiate and fit the t-SNE model.
from sklearn.manifold import TSNE
tsne_35 = TSNE(n_components = 2, perplexity = 35, verbose = 0 , n_iter = 1000).fit_transform(vec_array)

color_list = ["#003f5c", "#ffa600", "#f95d6a", "#a05195", "#ff7c43", "#2f4b7c", "#665191", "#d45087"]

data = []
for idx, i in  enumerate(df_grouped.account_category.unique()):
    x = tsne_35[df_grouped.account_category == i, 0]
    y = tsne_35[df_grouped.account_category == i, 1]
    
    data.append(go.Scatter(x = x, y = y, mode = 'markers', name = i, marker = dict(color = color_list[idx])))

layout = go.Layout(title = dict(text = 't-SNE Scatter: Original Labels', font = dict(size = 30)), legend = dict(font = dict(size = 15)))

fig = go.Figure(data = data, layout = layout)

py.iplot(fig)

Amazing! Our accounts seperate into distinct clusters that are more or less in line with the labels assigned by Warren and Linville! This suggests that we will be able to use KMeans or GMM to cluster accounts in an unsupervised way. 

One particularly interesting result is that there appears to be two distinct groups of Right Trolls that Warren and Linville did not discriminate between. By analyzing the behavior of these groups and others, we may be able to gain additional insight into Russian tradecraft and the overall objective of the Russian Twitter campaign.

Let's move on to our clustering algorithms!

### Kmeans

One of the challenges of using KMeans or any clustering algorithms is how to choose an appropriate K. We can use a metric called inertia to help aid us in choosing how many clusters we should specify for our data. 

But what is inertia and how does it work? Inertia is essentially a measure of clustering quality. That is, good clustering has tight clusters, and samples in each cluster are bunched together. Inertia measures how spread out clusters are and uses the distance from each sample to the centroid of its cluster in its calculation. 

A good rule of thumb is to choose the "elbow" in the inertia plot, or the inflection point where inertia begins to decrease at a slower rate.

Special shout-out to Hugo from Datacamp for teaching me about how inertia can be used to choose k, as well as laying the foundation for nearly everything that I know in python.

In [14]:
from sklearn.cluster import KMeans

inertias = []
clusters = list(range(1,11))

for k in clusters:
    
    model = KMeans(n_clusters = k)
    
    model.fit(vec_array)
    
    inertias.append(model.inertia_)

In [15]:
data = [go.Scatter(x = clusters, y = inertias, mode = 'lines', marker = dict(color = "#003f5c")),
        go.Scatter(x = clusters, y = inertias, mode = 'markers', marker = dict(color = "#2f4b7c", size = 9))]

layout = go.Layout(title = dict(text = 'Inertia Plot', font = dict(size = 30)), showlegend = False)

fig = go.Figure(data = data, layout = layout)

py.iplot(fig)

The inertia plot suggests that a k of five may be appropriate for our data. Let's go ahead and instantiate a KMeans model with k equal to five.

In [16]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5)

kmeans_labels = kmeans.fit_predict(vec_array)

In [17]:
color_list = ["#003f5c", "#ffa600", "#f95d6a", "#a05195", "#ff7c43", "#2f4b7c", "#665191", "#d45087"]

data = []
for idx, i in  enumerate(np.unique(kmeans_labels)):
    x = tsne_35[kmeans_labels == i, 0]
    y = tsne_35[kmeans_labels == i, 1]
    
    data.append(go.Scatter(x = x, y = y, mode = 'markers', name = str(i), marker = dict(color = color_list[idx])))

layout = go.Layout(title = dict(text = 't-SNE Scatter: KMeans Labels', font = dict(size = 30)), legend = dict(font = dict(size = 15)))

fig = go.Figure(data = data, layout = layout)

py.iplot(fig)

That looks great! I am actually surprised at how well KMeans seems to be performing on our data. 

KMeans is a discriminative algorithm, and can be thought of as a hard clustering technique. That is, a single element can only be assigned to one cluster. This may be problematic when analyzing text, as we are likely to see overlap between clusters and would naturally expect more ambiguity in our cluster assignments. For example, the ideological bent of a particular twitter acount may be unclear, and it may be difficult to place into one category or another based on the text alone. 

To account for uncertainty in cluster assignments, we could turn to a soft clustering method where we account for uncertainty in class assignment. For example, Gaussian Mixture Models (GMMs) are generative algorithms that provide a probabilistic way of doing soft clustering. 

GMMs can loosly be thought of as an extension of KMeans, except that instead of placeing centroids in random locations in space, we place probability distributions. We then use the Expectation Maximiazation algorithm to discover parameters for each probability distribution for our K sources, and move the distributions around until convergence.

GMMs allow us to calculate the probability that a sample belongs to a cluster. As such, each sample is assigned a probability that it belongs in each cluster. 

Let's go ahead and experiment with mixture models in the space below!

# Gaussian Mixture Models (Work in Progress)

I will be using Gaussian Mixture Models in the space below, although it is possible to use different kinds of probability distirbutions in mixture models. 

In [18]:
from sklearn.mixture import GaussianMixture as GMM

gmm = GMM(n_components=5).fit(vec_array)

gmm_labels = gmm.predict(vec_array)

In [19]:
color_list = ["#003f5c", "#ffa600", "#f95d6a", "#a05195", "#ff7c43", "#2f4b7c", "#665191", "#d45087"]

data = []
for idx, i in  enumerate(np.unique(gmm_labels)):
    x = tsne_35[gmm_labels == i, 0]
    y = tsne_35[gmm_labels == i, 1]
    
    data.append(go.Scatter(x = x, y = y, mode = 'markers', name = str(i), marker = dict(color = color_list[idx])))

layout = go.Layout(title = '2D representation of doc2vec clusters using GMM')

fig = go.Figure(data = data, layout = layout)

py.iplot(fig)

In [20]:
n_components = np.arange(1, 20)
models = [GMM(n, covariance_type='full', random_state=0).fit(vec_array)
          for n in n_components]

data = [go.Scatter(x = n_components, y = [m.bic(vec_array) for m in models], mode = 'lines', name = 'BIC'),
       go.Scatter(x = n_components, y = [m.aic(vec_array) for m in models], mode = 'lines', name = 'AIC')]

layout = go.Layout(xaxis = dict(title = 'n_components'))

py.iplot(data)

print('n_components associated with minimum AIC: ', 
      np.argmin([m.aic(vec_array) for m in models]) + 1)

n_components associated with minimum AIC:  5


In [21]:
gmm.predict_proba(vec_array)

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

Weird. I'm getting some results that I didn't expect, particularly how that class labels looks exactly the same as those from KMeans, and how the AIC/BIC chart looks. 

After thinking it over, I think that we may be falling victim to the curse of dimensionality. Since I am working with a relatively short and wide dataset, we might expect individual datapoints or even clusters of datapoints to be far apart in terms of distance. If this is the case, then the porbabilities that our model generates will be extremly low or extremly high, and this appears to be the case.

At any rate, I will be sticking with KMeans as I move forward. 

# Analysis

Now that we have completed our KMeans cluster assignments, it's time to see how they stack up against  Linville and Warrens labeling scheme.

In [22]:
df_grouped['kmeans_labels'] = kmeans_labels

df_grouped.head()

,author,account_category,parsed_content,kmeans_labels
0,4MYSQUAD,LeftTroll,"[dept, cover, cop, who, rap, female, officer, ...",1
1,AANTIRACIST,LeftTroll,"[donald, trump, head, meet, speaker, ryan, lit...",1
2,ABIGAILSSILK,HashtagGamer,"[.@midnight, the, dank, nug, returns, #StonedC...",2
3,ABIISSROSB,RightTroll,"[report, megyn, kelly, ’s, co, worker, outrage...",3
4,ABMERRLINS,RightTroll,"[break, new, demand, team, obama, face, crimin...",3


In [23]:
label_dict = {}

for row in df_grouped.iterrows():
    
    try: 
        label_dict[row[1].account_category].append(row[1].kmeans_labels)
    
    except:
        label_dict[row[1].account_category] = [row[1].kmeans_labels]

I want to compute the conditional probability of a cluster label given the label assigned by Linville and Warren. This is trickier than it seems, and I could not think of a better way to do this. 

In [24]:
choices = [0, 1, 2, 3, 4]

prob_df = pd.DataFrame()

for i in label_dict.keys():
    count = dict(Counter(label_dict[i]))
    length = sum(list(count.values()))

    prob_dict = {}

    for key in choices:
        try:
            prob_dict[key]= count[key]/length
        
        except:
            prob_dict[key]= 0
    
    row = pd.DataFrame(pd.Series(prob_dict)).T
    
    row.index = [i]
    
    prob_df = pd.concat([prob_df, row])

That was a lot of work!  We now have a dataframe of conditional probabilities. That is given a label by Warren and Linville, we have a conditional probability for each KMeans cluster label.  This is more clear if we view the dataframe. 

In [25]:
prob_df

,0,1,2,3,4
LeftTroll,0.000000,0.957627,0.000000,0.000000,0.042373
HashtagGamer,0.000000,0.031746,0.968254,0.000000,0.000000
RightTroll,0.000000,0.013453,0.000000,0.466368,0.520179
NewsFeed,0.933333,0.000000,0.000000,0.000000,0.066667
NonEnglish,0.000000,0.285714,0.000000,0.000000,0.714286
Commercial,0.200000,0.000000,0.800000,0.000000,0.000000
Unknown,0.000000,1.000000,0.000000,0.000000,0.000000


The table above indicates that for the most part, our clusters align pretty well with Warren and Linville's labels. Let's go ahead and visualize this. 

In [26]:
prob_df_t = prob_df.T

colors = ["#003f5c", "#58508d", "#bc5090", "#ff6361", "#ffa600"]
data = []
for idx, label in enumerate(list(prob_df_t.columns)[:6]):  
    data.append(go.Bar(x = list(prob_df_t.index), y = list(prob_df_t[label]), 
                       marker =  dict(color = colors), opacity = 0.9))
    
fig = tools.make_subplots(rows=2, cols=3, subplot_titles=('LeftTroll', 'HashtagGamer','RightTroll', 
                                                          'NewsFeed', 'NonEnglish', 'Commercial'))
fig.append_trace(data[0], 1, 1)
fig.append_trace(data[1], 1, 2)
fig.append_trace(data[2], 1, 3)
fig.append_trace(data[3], 2, 1)
fig.append_trace(data[4], 2, 2)
fig.append_trace(data[5], 2, 3)

fig['layout'].update(height=700, 
                     title= dict(text = 'Conditional Probability of KMeans Assignment Given a Label', 
                                 font = dict(size = 27)),
                     showlegend = False)

for i in range(1, 7):
    fig['layout']['yaxis' + str(i)].update(range = [0, 1], nticks = 10)


py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]



In [27]:
df_sampled = df_sampled.merge(df_grouped[['author', 'kmeans_labels']], 
                 left_on = 'author', right_on = 'author')